# Set Up based on M3

dataset: Toys and Games 5-core 

In [1]:
import json

In [19]:
dataset = []
with open('10_toy_games.txt') as json_file:
    line = json_file.readline()
    while line:
        dataset.append(json.loads(line))
        line = json_file.readline()

In [22]:
#@title Generic imports
import numpy as np
from gym import spaces
import matplotlib.pyplot as plt
#from scipy import stats

In [23]:
#@title RecSim imports
from recsim import document
from recsim import user
from recsim.choice_model import MultinomialLogitChoiceModel
from recsim.simulator import environment
from recsim.simulator import recsim_gym

# A Document Model

# TODO need to extract features from json and combine in a separate function during sampling.

In [26]:
class Document(document.AbstractDocument):
    def __init__(self, doc_id, features):
        self.features = features
        # doc_id is an integer representing the unique ID of this document
        super(Document, self).__init__(doc_id)

    def create_observation(self):
        return np.array([self.features])

    @staticmethod
    def observation_space():
        return spaces.Box(shape=(1,), dtype=np.float32, low=-np.inf, high=np.inf)

    def __str__(self):
        return "Document {} with features {}.".format(self._doc_id, self.features)

In [45]:
class DocumentSampler(document.AbstractDocumentSampler):
    def __init__(self, data, doc_ctor=Document):
        super(DocumentSampler, self).__init__(doc_ctor)
        self._doc_count = 0
        self._data = data

    def sample_document(self):
        #sampled_idx = np.random.choice(range(len(self._data)))
        doc_features = {}
        doc_features['doc_id'] = self._doc_count
        doc_features['features'] = self._data[self._doc_count]['overall']
        self._doc_count += 1
        return self._doc_ctor(**doc_features)

In [46]:
sampler = DocumentSampler(data=dataset)
for i in range(5): print(sampler.sample_document())
d = sampler.sample_document()
print("Documents have observation space:", d.observation_space(), "\n"
      "An example realization is: ", d.create_observation())

Document 0 with features 5.0.
Document 1 with features 4.0.
Document 2 with features 5.0.
Document 3 with features 5.0.
Document 4 with features 4.0.
Documents have observation space: Box(1,) 
An example realization is:  [3.]


# User Model